In [ ]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import random
from PIL import Image
import os
import scipy.io

import sys
sys.path.append('../python')
import caffe

plt.rcParams['image.cmap'] = 'gray'
plt.rcParams['image.interpolation'] = 'nearest'

### Run inference with trained Yearbook models on different datasets

In [ ]:
# Helper methods for running inference in Caffe

def feed_net(net, in_):
    net.blobs['data'].reshape(1, *in_.shape)
    net.blobs['data'].data[...] = in_
    
def run_net(net, in_):
    feed_net(net, in_)
    net.forward()
    return np.squeeze(net.blobs['yrbook'].data)
    #return np.squeeze(net.blobs['fc8'].data)

def preprocess(im, mean):
    """
    Preprocess loaded image (by load_image) for Caffe:
    - cast to float
    - switch channels RGB -> BGR
    - subtract mean
    - transpose to channel x height x width order
    """
    in_ = im.astype(np.float32)
    in_ = in_[:, :, ::-1]
    in_ -= np.array(mean)
    in_ = in_.transpose((2, 0, 1))
    return in_

def load_label(idx):
    """
    Compute the label from the filename
    """
    min_year = 28 
    year = idx.split('_')[0]
    if year[0] == str(1): # 20th century
        label = int(year[-2:]) - min_year
    else: # 21st century
        label = int(year[-2:]) + (100 - min_year)
    return label

def label_to_year(label):
    ''' Convert category labels back to years'''
    return label + 1928

In [ ]:
# Config
caffe.set_device(0)
caffe.set_mode_gpu()
exp = EXP
it = ITERATION

# load deploy net because we want to run the net on data that is not configured in the Python data layer
proto = '../output/deploy.prototxt'.format(exp)
weights = '../output/{}/snapshots/train_iter_{}.caffemodel'.format(exp, it)
net = caffe.Net(proto, weights, caffe.TEST)

In [ ]:
# Images should reside in DATADIR/images and be listed in a file 'test.txt'

data_dir = DATADIR 
women_test = open(os.path.join(data_dir, 'test.txt')).read().splitlines()
im_shape = (96,96)
mean = MEAN

preds = np.zeros(len(women_test))
gts = np.zeros(len(women_test))
diffs = np.zeros(len(women_test))

correct = 0    
for i, path in enumerate(women_test):
    img = np.array(Image.open('{}/images/{}'.format(data_dir, path)))
    img = scipy.misc.imresize(img, im_shape)
    pred = run_net(net, preprocess(img, mean))
    probvec = np.squeeze(net.blobs['dist'].data) # softmax distribution
    preds[i] = int(np.argmax(pred))
    gts[i] = load_label(path)
    if preds[i] == gts[i]:
        correct += 1
    diffs[i] = np.abs(preds[i] - gts[i])    

### Some simple metrics

In [ ]:
# Print other metrics
print 'Accuracy', correct / float(len(women_test))
print 'Avg Error', np.average(diffs)
print 'Median Error', np.median(diffs)
print 'Class. Accuracy Within 5 Years', len([d for d in diffs if d <= 5]) / float(len(women_test))
print 'Class. Accuracy Within 10 Years', len([d for d in diffs if d <= 10]) / float(len(women_test))

### Histogram of L1 Errors

In [ ]:
import seaborn as sns

from matplotlib import rc
plt.rcParams['figure.figsize'] = (10, 6)

fig, ax = plt.subplots()
plt.hist(diffs, 7, normed=0, alpha=1, color='grey')
plt.xlabel('Error in Years', labelpad=8)
plt.ylabel('Number of Portraits', labelpad=8)
print min(diffs)
print max(diffs)

for item in ([ax.title, ax.yaxis.label, ax.xaxis.label]):
    item.set_fontsize(20)
for item in ax.get_xticklabels() + ax.get_yticklabels():
    item.set_fontsize(18)
plt.savefig('L1-celeb-errors.pdf', bbox_inches='tight')
plt.show()
print sum(1 for x in diffs if x <=10)